<a href="https://colab.research.google.com/github/srivardhan-kondu/Explainable-Brain-Tumor-Detection-model-using-Masked-Autoencoders-and-Grad-CAM/blob/main/Explainable_Brain_Tumor_Detection_model_using_Masked_Autoencoders_and_Grad_CAM.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [20]:
import tensorflow as tf
import matplotlib.pyplot as plt


In [21]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [22]:
dataset_path = '/content/drive/MyDrive/brain_tumor_dataset'